
<img align="right" width="125" src="https://www.ou.nl/documents/40554/3255217/Logo_OU.jpg"/>

<br>


<center> <font size ="6" color='red'> Machine Learning Training Dashboard </font></center>
<br> 

<br>

Model Development dashboard 


In [ ]:
from IPython.display import clear_output
from IPython import display
from data_cleaning_view import get_data_cleaning_tab
from data_processing_view import get_data_processing_tab
from data_selection_view import get_data_selection_tab
from ipywidgets import *
from data_cleaning_manager import *
from data_processing_manager import *
from data_selection_manager import *
from predictive_modeling_manager import *
from predictive_modeling_view import get_predictive_modeling_tab
import settings
import warnings

warnings.filterwarnings("ignore")
settings.init()

# def init_tabs():


if settings.online_version:
    user = "muratfirat78"
    repo = "CPP_Datasets"

    if os.path.isdir(repo):
        !rm -rf {repo}

    !git clone https://github.com/{user}/{repo}.git
    %cd /content/{repo}

tab_1 = get_data_selection_tab()
tab_2 = get_data_cleaning_tab()
tab_3 = get_data_processing_tab()
tab_4 = get_predictive_modeling_tab()
# init_tabs()
tab_set = widgets.Tab([tab_1,tab_2,tab_3,tab_4])
tab_set.set_title(
    0, 'Data Selection')

tab_set.set_title(1, 'Data Cleaning')
tab_set.set_title(2, 'Data Processing')
tab_set.set_title(3, 'Predictive Modeling')

tab_set
